This notebook describes the temporal fusion transformers [@lim2021temporal] architecture, and ports it over to keras 3 while making some punctual improvements, including bringing the notation closer to the one in the paper.

The original repository is [here](https://github.com/google-research/google-research/tree/master/tft).

In [1]:
#| output: false

from __future__ import annotations

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import keras
from keras import layers

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from datetime import timedelta
from dateutil.relativedelta import relativedelta
from fastcore import docments
from nbdev.showdoc import show_doc
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

2024-01-14 14:16:22.687447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Introduction

The main characteristics of TFT that make it interesting for nowcasting or forecasting purposes are:

- **multi-horizon forecasting**: the ability to output, at each point in time $t$, a sequence of forecasts for $t+h, h > 1$
- **quantile prediction**: each forecast is accompanied by a quantile band that communicates the amount of uncertainty around a prediction
- **flexible use of different types of inputs**: static inputs (akin to fixed effects), historical input and known future input (eg, important holidays, years that are known to have major sports events such as Olympic games, etc)
- **interpretability**: the model learns to select variables from the space of all input variables to retain only those that are globally meaningful, to assign attention to different parts of the time series, and to identify events of significance

## Main innovations

The present model includes the following innovations:

- **Multi-frequency input**

- **Context enhancement from lagged target**: the last known values of the target variable are embedded (bag of observations), and this embedding is used similar to the static context enhancement as a starting point for the cell value in the *decoder* LSTM.

# Preparing the data

The functions below will be tested with simulated and real data. The former helps to illustrate issues like dimensions and overall behaviour of a layer, whereas the latter will demonstrate on a real setting in economics how the input and output data relate to one another.

More specifically, the real data used will be a daily nowcasting exercise of monthly inflation. Note that the models will not necessarily perform well, since their use here is for illustration purposes and thus they are not optimised. Also, the dataset is not an ideal one for nowcasting: other variables could also be considered.

## Download economic data

This is a panel dataset. In addition to the time dimension, it can contain any number of categorical dimensions - for example, combine country and sector.

In [ ]:
#| warning: false

df_all = pd.read_csv("data/nowcast_dataset_complete_Dec-08-2023.csv")
df_all['index'] = pd.to_datetime(df_all['index'])

## Prepare data

In [ ]:
countries = ['AU', 'CA', 'CH', 'DE', 'FR', 'GB', 'IN', 'JP', 'US']

filter_freq_d = df_all['frequency'] == 'd'
filter_cty = df_all['country'].isin(countries)
filter_dates = df_all['index'] >= '1980-01-01'

In [ ]:
df_input = df_all[filter_freq_d & filter_cty & filter_dates].copy()
df_input.drop (['Unnamed: 0', 'frequency'], axis=1, inplace=True)
df_input.set_index(['index', 'country'], inplace=True) 
df_input = df_input.unstack('country')
df_input.columns = ['__'.join(col).strip() for col in df_input.columns.values]
df_input.dropna(how='all', inplace=True)

In [ ]:
target_var = 'CPIh'
df_target = df_all.loc[
    (df_all['frequency'] == 'm') & (df_all['country'].isin(countries)) & filter_dates,
    ['index', 'country'] + [target_var]
] \
    .set_index(['index', 'country']) \
    .unstack('country') \
    .droplevel(0, axis=1) \
    .dropna()

In [ ]:
df_target_12m_pct = (100 * df_target.pct_change(12))
df_target_1m_pct = (100 * df_target.pct_change(1))

In [ ]:
#| fig-align: center

ax = df_target_12m_pct.plot(figsize=(20, 6))
ax.axhline(y=0, color='black')
ax.set_title("Inflation", fontsize=16, fontstyle='italic')
plt.show()

## Date features

In addition to data provided by the user, the model automatically loads categorical features related to each date.

In [ ]:
def date_features(
    date_range, # Range of dates for which to create date features
    is_monthly:bool=False # Is the date measured at the monthly frequency?
)->pd.DataFrame: # Categorical date features
    "Categorical features for each day in a range of dates"
    if is_monthly:
        return pd.DataFrame({
        'Month of Year': date_range.month
    })
    else:
        return pd.DataFrame({
            'Day of Week': date_range.dayofweek + 1, # This is the only date feature with zeros, which are masked out
            'Day of Month': date_range.day,
            'Day of Year': date_range.dayofyear,
            'Week of Month': (date_range.day - 1) // 7 + 1,
            'Week of Year': pd.Index(date_range.isocalendar().week).astype('int32'),
            'Month of Year': date_range.month
        })


In [ ]:
#| output: asis
#| echo: false

show_doc(date_features, title_level=3)

In [ ]:
#| code-fold: show

date_features(df_input.index)[:10]

### Combining the date features in the main data

In [ ]:
date_feat = date_features(df_input.index)
date_feat.index = df_input.index
df_input = pd.concat([df_input, date_feat], axis=1)

## Splitting the dataset

This step needs to be done somewhat differently than normal datasets: In the validation and test periods, the input data can very well be from the previous chunk (training and validation, respectively).

In practice, this means that we just need to split the **dates** at which the nowcasting will take place. Then, a data loading function (see @sec-dataloaders) can read from the whole time series to look back from that date as needed according to the desired time window.

In [ ]:
start_date = df_input.index.min()
end_date = df_input.index.max()
cutoff_date_train = pd.to_datetime('2005-01-01')
cutoff_date_valid = pd.to_datetime('2020-01-01')

dates_train = pd.date_range(start_date, cutoff_date_train - timedelta(days=1))
dates_valid = pd.date_range(cutoff_date_train, cutoff_date_valid - timedelta(days=1))
dates_test = pd.date_range(cutoff_date_valid, end_date)

Now all input variables that are not available in the training dataset are removed, and only those with at least some information are kept.

In [ ]:
keep_cols = df_input.loc[min(dates_train):max(dates_train)].dropna(axis=1, how='all').columns
df_input = df_input[keep_cols]

## Identifying continuous and categorical variables {#sec-contcat}

 The model distinguishes continuous from categorical variables if the user does not provide a list of variable names in a simple (simplistic) way: integer-valued variables that start with one are considered categorical, all other are continuous.

The criteria that categorical variables start with one is to ensure that the user does not unwarrantedly pass on categorical variables with zero, since zeros are considered to be a padding for variable-length input data.

For variables that are naturally valued in integers, such as the count of number of firms, etc, the user can either ensure there is a zero amongst the integer at any time of the **training** input data, or convert these values to floating numbers. Another alternative that might be relevant in some cases is to use the natural logarithm of that data.

In [ ]:
int_cols = df_input.select_dtypes(include=['int']).columns
float_cols = df_input.select_dtypes(include=['float']).columns

# Columns that are float but actually contain integer values starting with one
cat_cols = []

for col in int_cols:
    if min(df_input[col]) > 0:
        cat_cols.append(col)

for col in float_cols:
    if (df_input[col] % 1 == 0).all() and min(df_input[col]) > 0:  # Check if the fractional part is 0 for all values and the lowest integer is 1
        cat_cols.append(col)

cont_cols = [c for c in df_input.columns if c not in cat_cols]

assert len(cont_cols) + len(cat_cols) == df_input.shape[1]

Further, the categorical variables require a dictionary that indicates the cardinality of each variable.

## Scaling the continuous variables

The input series need to be scaled, according to the training data mean and standard deviation.

The target series will not be scaled because it is already a small number close to zero that is not exploding in nature.

In [ ]:
#| warning: false

scl = StandardScaler()
scl.fit(df_input.loc[dates_train.min():dates_train.max(), cont_cols])
df_input_scl = pd.DataFrame(
    scl.transform(df_input[cont_cols]),
    index=df_input.index,
    columns=cont_cols
    )
df_input_scl = pd.concat([df_input_scl, date_feat], axis=1)

assert df_input_scl.shape == df_input.shape

## Cardinality of categorical variables

Each categorical variable has its own cardinality. This value is important when creating the embedding layer for each variable; see @sec-input.

In [ ]:
cardin_hist = {c: len(df_input_scl[c].unique()) + 1 for c in cat_cols}

cardin_hist

The cardinality of the static variable(s) must also be included:

In [ ]:
cardin_stat = dict(Countries=len(df_target.columns) + 1)

The country list also requires an encoding/decoding dictionary for subsequent analyses.

In [ ]:
country_enc_dict = {cty: idx + 1 for idx, cty in enumerate(df_target.columns)}
country_dec_dict = {idx: cty for cty, idx in country_enc_dict.items()}

## Dealing with missing data


Missing data is dealt with by replacing `NaN`s in the input data with zeros. This has two effects:

* it prevents embedding categorical variables since zeros are masked out

* for the continuous data, the input layer weights do not pick up any information, and the constant (or "bias" in machine learning language) is responsible for conveying any information to subsequent neurons.

A more sophisticated approach would be to estimate missing data based on other contemporaneous and past data. For simplicity, this approach will not be followed in this example.

Obviously this step needs to be done after the input data is scaled, otherwise the zeros would be wrongly contributing to the mean and standard deviation values.

In [ ]:
df_input_scl.fillna(0, inplace=True)

Finally, we remove the input months for which there is no inflation data (eg, due to the one-month growth calculation):

In [ ]:
df_input_scl = df_input_scl[df_target_1m_pct.dropna().index.min():]

## Splitting the data

Now we separate the training, validation and test data.

In [ ]:
df_input_train = df_input_scl[:dates_train.max()]
df_input_valid = df_input_scl[dates_train.max():dates_valid.max()]
df_input_test = df_input_scl[dates_valid.max():dates_test.max()]

df_input_train.shape[0], df_input_valid.shape[0], df_input_test.shape[0]

## Data loaders {#sec-dataloaders}

Ideally a data loader should:

* create a pair of input/target data

* the input data should contain:

    * continuous data from all countries

    * categorical date features

    * categorical feature of the country (ie, which country it is)

    * known future inputs

On the known future inputs: those will be essentially the categorical date features, broadcasted until the end of the desired month to be nowcasted (ie, either the current month or a future one). The nowcast will then be the value of inflation at the end of the "as-of" month.

> Note: so far, the only known future data used by the model are the categorical features from the dates up to the last day of the nowcasted/forecasted month. However, an important known future data for inflation are the central bank policy variables. These are not yet dealth with in this code, but a future version should incorporate an intelligent way for users to input a vector of policy variables, with the dates up until which they would be known. This could be done in a separate DataFrame with policy variables, which would arguably facilitate working with this data separately from all others.

In [ ]:
def sample_nowcasting_data(
    df_daily_input:pd.DataFrame,
    df_target:pd.DataFrame,
    min_context:int=10, # minimum context length in number of daily observations
    context_length:int=120, # context length in number of daily observations (leads to padding if not reached in sampling)
    num_months:int=12 # number of months (1 is current month)
):

    # first step: determine the valid dates for sampling
    # from the left, they should allow at least `context_length` up until the sampled date
    # from the right, they should allow enough room to retrieve all of the target inflation months
    # only the dates "in the middle" can be sampled
    all_dates = df_daily_input.index
    earliest_poss_date = all_dates[min_context]

    delta_latest_month = num_months - 1
    latest_poss_date = all_dates.max() - relativedelta(months=delta_latest_month) + pd.offsets.MonthEnd(0)

    dates_for_sampling = df_daily_input.loc[earliest_poss_date:latest_poss_date].index

    # sample a random date, context length and country
    sampled_day = pd.to_datetime(np.random.choice(dates_for_sampling))
    sampled_ctl = np.random.randint(low=min_context, high=context_length) \
        if min_context < context_length \
        else context_length
    X_cat_stat = np.random.randint(low=1, high=len(country_enc_dict)+1, size=(1,))

    # create the historical observed data
    earliest_date = sampled_day - relativedelta(days=sampled_ctl)
    df_hist = df_daily_input.loc[earliest_date:sampled_day]
    if df_hist.shape[0] < context_length:
        df_pad = pd.DataFrame(np.zeros((context_length-df_hist.shape[0], df_hist.shape[1])))
        df_pad.columns = df_hist.columns
        df_hist = pd.concat([df_pad, df_hist], axis=0, ignore_index=True)
    X_cont_hist = df_hist[cont_cols].values
    X_cat_hist = df_hist[cat_cols].values

    # create the future known data: month of the year
    # note: any other known future information of interest should be included here as well
    # eg: mon pol committee meeting dates, months of major sports events, etc
    # anything that could influence inflation dynamics
    target_month = (sampled_day + relativedelta(months=num_months)).replace(day=1) \
        if num_months == 1 \
        else [(sampled_day + relativedelta(months=d)).replace(day=1) for d in range(num_months)]
    X_fut = date_features(pd.DataFrame(index=target_month).index, is_monthly=True).values

    # create the target variables
    y = df_target.loc[target_month, country_dec_dict[int(X_cat_stat)]].values
    
    X_cat_stat = keras.ops.expand_dims(X_cat_stat, axis=1)
    return [X_cont_hist, X_cat_hist, X_fut, X_cat_stat], y

In [ ]:
show_doc(sample_nowcasting_data, title_level=3)

Note that `min_context` not necessarily number of days because of weekends, etc.

In practice, this means that the sampled data almost always needs to be padded, even when `min_context` is equal to `context_length`. See the example below.

In [ ]:
[X_cont_hist, X_cat_hist, X_fut, X_cat_stat], y = sample_nowcasting_data(
    df_daily_input=df_input_scl.loc[:dates_train.max()],
    df_target=df_target_1m_pct,
    min_context=120,
    context_length=120
)

X_cont_hist.shape, X_cat_hist.shape, X_fut.shape, X_cat_stat.shape, y.shape

In [ ]:
pd.DataFrame(X_cont_hist).plot(legend=False)

In [ ]:
def prepare_data_samples(
    n_samples:int=1000,
    **kwargs
):
    "Transforms the dataset from tabular format to a dataset used for training the TFT model."
    X_cont_hist = []
    X_cat_hist = []
    X_fut = []
    X_cat_stat = []
    y = []
    for i in tqdm(range(n_samples)):
        [indiv_cont_hist, indiv_cat_hist, indiv_fut, indiv_cat_stat], indiv_y = sample_nowcasting_data(**kwargs)
        X_cont_hist.append(indiv_cont_hist)
        X_cat_hist.append(indiv_cat_hist)
        X_fut.append(indiv_fut)
        X_cat_stat.append(indiv_cat_stat)
        y.append(indiv_y)
    
    X_cont_hist = keras.ops.stack(X_cont_hist, axis=0)
    X_cat_hist = keras.ops.stack(X_cat_hist, axis=0)
    X_fut = keras.ops.stack(X_fut, axis=0)
    X_cat_stat = keras.ops.stack(X_cat_stat, axis=0)
    y = keras.ops.stack(y, axis=0)

    return [X_cont_hist, X_cat_hist, X_fut, X_cat_stat], y

In [ ]:
show_doc(prepare_data_samples, title_level=3)

This function serves the purpose to structure the data in a way that the TFT can ingest, while taking advantage of the number of combinations of sampled data X context size to create a larger dataset.

In [ ]:
train_data = prepare_data_samples(
    n_samples=1000,
    df_daily_input=df_input_scl.loc[:dates_train.max()],
    df_target=df_target_1m_pct
)

valid_data = prepare_data_samples(
    n_samples=1000,
    df_daily_input=df_input_scl.loc[dates_train.max():dates_valid.max()],
    df_target=df_target_1m_pct
)

In [ ]:
[X_cont_hist, X_cat_hist, X_cat_fut, X_cat_stat], y = train_data
X_cont_hist.shape, X_cat_hist.shape, X_cat_fut.shape, X_cat_stat.shape, y.shape

In [ ]:
pd.DataFrame(X_cont_hist[0, Ellipsis]).plot(legend=False)

Note that the argument `max_context_length_days` cannot promise the user to reach the actual number, since there might be weekends or other dates without any information.

For models that will nowcast/forecast more than one month, the argument `delta_month` needs to be a list or an iterator as `range()`:

Now the months for which to forecast are February (the current month of the sampled day), March and April, corresponding to the three months indicated in the argument.

All historical time series in the same batch have the same length. This length varies beetween batches.

# Architecture

First, common notation is introduced, and then individual components are presented. At the end of this section, the whole model is put together.

## Notation

* unique entities: $i \in (1, \dots\, I)$
* time periods $t \in [0, T_i]$
  * $k \geq 1$ lags
  * $h \geq 1$ forecasting period
* set of entity-level static covariates: $s_i \in \mathbf{R}^{m_s}$
* set of temporal inputs: $\chi_{i, t} \in \mathbf{R}^{m_\chi}$
  * $\chi_{i,t} = [z_{i,t}, x_{i,t}]$
    * $z_{i,t} \in \mathbf{R}^{m_z}$ are historical inputs
    * $x_{i,t} \in \mathbf{R}^{m_z}$ are a priori known inputs (eg, days of the week or years that have major sports events)
  * $m_\chi$ is the number of total input variables, where $m_\chi = m_z + m_x$
* target scalars: $y_{i,t}$
  * $\hat{y}_{i,t,q} = f_q(y_{i,t-k:t}, z_{i,t-k:t}, x_{i,t-k:t+h}, s_i)$
* hidden unit size (common across all the TFT architecture for convenience): $d_{\text{model}}$
* transformed input of $j$-th variable at time $t$: $\xi_t^{(j)} \in \mathbf{R}^{d_{\text{model}}}$
  * $\Xi_t = [\xi_t^{(1)}, \dots, \xi_t^{(m_\chi)}]$

## Example data

All real data examples below use the same data from @sec-real_data.

## Dense layer

One of the fundamental units in the TFT network is the dense layer:

$$
\mathbb{Y} = \phi(\mathbf{W} x + \mathbf{b}),
$$ {#eq-dense}

where $x$ is the input data and $\mathbb{Y}$ is its output, $\phi$ is an activation function (when it is applied), $\mathbf{W} \in \mathbf{R}^{(d_{\text{size}} \times d_{\text{inputs}})}$ is a matrix of weights and $\mathbf{b} \in \mathbf{R}^{d_{size}}$ is a vector of biases.

## Input data transformations {#sec-input}

> Transforms all input variables into a latent space

All input data, regardless if historical, static or future, are transformed into a feature representation with dimension $d_{\text{model}}$. In other words, $\chi_{t}^{(j)}$, variable $j$'s observation at each time period $t$, undergoes an injective mapping $f^{(j)} : \mathbb{R} \to \mathbb{R}^{d_{\text{model}}}$ for continuous data and $f^{(j)} : \mathbb{N} \to \mathbb{R}^{d_{\text{model}}}$ for categorical data.

If the variable $\chi_{t}^{(j)}$ is continuous, this transformation is done by $d_{\text{model}}$ linear regressions, the coefficients of which are determined as part of the neural network training:

$$
\xi_t^{(j)} = \mathbf{W}^{(j)} \chi_{t}^{(j)} + \mathbf{b}^{(j)},
$$

where $\xi_t^{(j)}, \mathbf{W}^{(j)}, \mathbf{b}^{(j)} \in \mathbb{R}^{d_{\text{model}}}$. Note that $\mathbf{W}^{(j)}, \mathbf{b}^{(j)}$ are the same for variable $j$ at all time periods (ie, the layer is time-distributed).

Conversely, if the $j^{\text{th}}$ variable is categorical, then the transformation is an embedding. Each embedding layer requires a specific number of different categories, ie the cardinality of the categorical variable. This cardinality is assumed to be stable or decreasing outside of the training period; otherwise a new category would appear at testing time for which the model has not learned an embedding.

In any case, it is this embedded data, $\xi_t^{(j)}$, that is used in all the next steps of the TFT network.

See @sec-contcat for details of how the model determines which variables are continuous or categorical.

In [ ]:
#| code-fold: hide

class MultiInputContEmbedding(keras.Layer):
    def __init__(
        self, 
        d_model:int, # Embedding size, $d_\text{model}$
        **kwargs
    ):
        "Embeds multiple continuous variables, each with own embedding space"
        
        super(MultiInputContEmbedding, self).__init__(**kwargs)
        self.d_model = d_model

    def build(self, input_shape):
        super(MultiInputContEmbedding, self).build(input_shape)

        # input_shape: (batch_size, num_time_steps, num_variables)
        num_variables = input_shape[-1]
        
        self.kernel = self.add_weight(
            shape=(num_variables, self.d_model),
            initializer='uniform',
            name='kernel'
        )

        self.bias = self.add_weight(
            shape=(self.d_model,),
            initializer='zeros',
            name='bias'
        )
    
    def call(
        self,
        inputs # Data of shape: (batch size, num time steps, num variables)
    ):     
        "Output shape: (batch size, num time steps, num variables, d_model)"

        # Applying the linear transformation to each time step
        output = keras.ops.einsum('bti,ij->btij', inputs, self.kernel)
        output += self.bias
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[1], input_shape[2], self.d_model


In [ ]:
#| output: asis
#| echo: false

show_doc(MultiInputContEmbedding.__init__, title_level=3)

In [ ]:
#| output: asis
#| echo: false

show_doc(MultiInputContEmbedding.call, title_level=3)

In [ ]:
#| code-fold: show

xi_cont_hist = MultiInputContEmbedding(d_model=16)(X_cont_hist)

print("Input data shape: (batch size / num time steps / num variables)", X_cont_hist.shape)
print("Encoded data shape: (batch size / num time steps / num variables / d_model)", xi_cont_hist.shape)

In [ ]:
#| code-fold: hide

class MultiInputCategEmbedding(keras.Layer):
    def __init__(
        self,
        d_model:int, # Embedding size, $d_\text{model}$
        cardinalities:dict, # Variable: cardinality in training data
        **kwargs
    ):
        "Embeds multiple categorical variables, each with own embedding function"
        super(MultiInputCategEmbedding, self).__init__(**kwargs)
        self.d_model = d_model
        self.cardinalities = cardinalities
        
    def build(self, input_shape):
        super(MultiInputCategEmbedding, self).build(input_shape)
        if len(self.cardinalities.keys()) != input_shape[-1]:
            raise ValueError("`cardinalities` should have as many elements as the input data's variables.")
        
        self.embed_layer = {var:
            keras.Sequential([
                layers.Embedding(
                    input_dim=cardin,
                    output_dim=self.d_model,
                    mask_zero=True,
                    name="Input_embed_" + var.replace(" ", "_")
                )
            ]) for var, cardin in self.cardinalities.items()
        }
        super(MultiInputCategEmbedding, self).build(input_shape)

    def call(
        self,
        inputs # Data of shape: (batch size, num time steps, num variables)
    ):
        "Output shape: (batch size, num time steps, num variables, d_model)"
        embeds = [
            self.embed_layer[var](inputs[:,:,idx])
            for idx, var in enumerate(self.cardinalities.keys())
        ]
        return keras.ops.stack(embeds, axis=2) # keras.ops.concatenate(embeds, axis=-1)


In [ ]:
#| output: asis
#| echo: false

show_doc(MultiInputCategEmbedding.__init__, title_level=3)

In [ ]:
#| output: asis
#| echo: false

show_doc(MultiInputCategEmbedding.call, title_level=3)

Note that this function masks out the data whenever categories are set to zero. This is to ensure that the model can take in variable-sized inputs. Because of this, the model requires inputted categorical data to be added 1 whenever zero is a possible category.

> Note to self: I don't like the approach above where we rely implicitly on the input data's ordering to extract the cardinality. But this is a practical way to get things going. It should be changed to a more robust in the future.

In [ ]:
#| code-fold: show

xi_cat_hist = MultiInputCategEmbedding(
    d_model=16, 
    cardinalities=cardin_hist
)(X_cat_hist)

print("Input data shape: (batch size / num time steps / num variables)", X_cat_hist.shape)
print("Encoded data shape: (batch size / num time steps / num variables / d_model)", xi_cat_hist.shape)

In [ ]:
#| code-fold: show

xi_cat_fut = MultiInputCategEmbedding(
    d_model=16, 
    cardinalities={'Month of Year': 12 + 1} # hardcoded because it is always just the months in an year
)(X_cat_fut)

print("Input data shape: (batch size / num time steps / num variables)", X_cat_fut.shape)
print("Encoded data shape: (batch size / num time steps / num variables / d_model)", xi_cat_fut.shape)
print(keras.ops.any(keras.ops.isnan(xi_cat_fut)))

In [ ]:
keras.ops.expand_dims(X_cat_stat, axis=-1).shape

In [ ]:
#| code-fold: show

xi_stat = MultiInputCategEmbedding(
    d_model=16, 
    cardinalities=cardin_stat
)(X_cat_stat)

print("Input data shape: (batch size / num time steps / num variables)", X_cat_stat.shape)
print("Encoded data shape: (batch size / num time steps (=one) / num variables / d_model)", xi_stat.shape)
print(keras.ops.any(keras.ops.isnan(xi_stat)))

In [ ]:
#| code-fold: hide

class InputTFT(keras.Layer):
    def __init__(
        self,
        d_model:int=16, # Embedding size, $d_\text{model}$
        **kwargs
    ):
        "Input layer for the Temporal Fusion Transformer model"
        super(InputTFT, self).__init__(**kwargs)
        self.d_model = d_model
        
        self.flat = layers.Flatten()
        self.concat = layers.Concatenate(axis=2)
    
    def build(self, input_shape):
        self.cont_hist_embed = MultiInputContEmbedding(
            self.d_model,
            name="embed_continuous_historical_vars"
        )
        self.cat_hist_embed = MultiInputCategEmbedding(
            self.d_model, 
            cardinalities=cardin_hist,
            name="embed_categ_historical_vars"
        )
        self.cat_fut_embed = MultiInputCategEmbedding(
            self.d_model, 
            # Note below the same categorical variables are just the months in an year. 
            # This situation may not apply to all cases.
            # More complex models using other categorical future known data might require
            # another cardinalities dictionary.
            cardinalities={'Month of Year': 12},
            name="embed_categ_knownfuture_vars"
        )
        self.cat_stat_embed = MultiInputCategEmbedding(
            self.d_model, 
            cardinalities=cardin_stat,
            name="embed_categ_static_vars"
        )
        super(InputTFT, self).build(input_shape)

    def call(
        self, 
        # List of data with shape: [(batch size, num hist time steps, num continuous hist variables), (batch size, num hist time steps, num categorical hist variables), (batch size, num static variables), (batch size, num future time steps, num categorical future variables)]
        input:list 
    ):
        """List of output with shape: [
            (batch size, num hist time steps, num historical variables, d_model),
            (batch size, num future time steps, num future variables, d_model)
            (batch size, one, num static variables, d_model),
        ]"""
        cont_hist, cat_hist, cat_fut, cat_stat = input
        if len(cat_stat.shape) == 2:
            cat_stat = keras.ops.expand_dims(cat_stat, axis=-1)

        cont_hist = self.cont_hist_embed(cont_hist)
        #cont_hist = keras.ops.swapaxes(cont_hist, axis1=2, axis2=3)

        cat_hist = self.cat_hist_embed(cat_hist)
        #cat_hist = self.flat(cat_hist)
            
        cat_fut = self.cat_fut_embed(cat_fut)
        #cat_fut = self.flat(cat_fut)
        
        cat_stat = self.cat_stat_embed(cat_stat)
        #cat_stat = self.flat(cat_stat)

        # (batch size / (num time steps * (num historical + future variables) + num static variables) * embedding size)
        hist = self.concat([cont_hist, cat_hist])
        
        return hist, cat_fut, cat_stat

In [ ]:
#| output: asis
#| echo: false

show_doc(InputTFT.__init__, title_level=3)

In [ ]:
#| output: asis
#| echo: false

show_doc(InputTFT.call, title_level=3)

In [ ]:
#| code-fold: show

xi_hist, xi_fut, xi_stat = InputTFT()([X_cont_hist, X_cat_hist, X_cat_fut, X_cat_stat])
print("Encoded historical data shape: (batch size / num hist time steps / num historical variables / d_model)", xi_hist.shape)
print("Encoded historical data shape: (batch size / num future time steps / num future variables / d_model)", xi_fut.shape)
print("Encoded historical data shape: (batch size / one / num static variables / d_model)", xi_stat.shape)

**From now on, whenever relevant the examples with real data will use the $\xi$ elements created above.**

The following simplistic model shows how the input layer is used. The goal is to highlight how the data is inputted into a TFT model, by not focusing on its complexity just now.

First, it takes up the data. Then, in this simplified model it flattens all inputs and uses a dense layer connected to all embeddings at all time points to output the forecast.

## Skip connection

> Adds inputs to layer and then implements layer normalisation

$$
\text{LayerNorm}(a + b),
$$ {#eq-skip}

for $a$ and $b$ tensors of the same dimension and $\text{LayerNorm}(\cdot)$ being the layer normalisation (@ba2016layer), ie subtracting $\mu^l$ and dividing by $\sigma^l$ defined as:

$$
\mu^l = \frac{1}{H} \sum_{i=1}^H n_i^l \quad \sigma^l = \sqrt{\frac{1}{H} \sum_{i=1}^H (n_i^l - \mu^l)^2},
$$ {#eq-layernorm}

with $H$ denoting the number of $n$ hidden units in a layer $l$.

* Adding a layer's inputs to its outputs is also called "skip connection"
* The layer is then normalised [@ba2016layer] to avoid having the numbers grow too big, which is detrimental for gradient transmission
  * Layer normalisation uses the same computation both during training and inference times, and is particularly suitable for time series

## Gated linear unit (GLU)

> Linear layer that learns how much to gate vs let pass through

Using input $\gamma \in \mathbb{R}^{d_{\text{model}}}$ and the subscript $\omega$ to index weights, 

$$
\text{GLU}_{\omega}(\gamma) = \sigma(W_{4, \omega} \gamma + b_{4, \omega}) \odot (W_{5, \omega} \gamma + b_{5, \omega}),
$$ {#eq-GLU}

where $\mathbf{W} \in \mathbf{R}^{(d_{\text{model}} \times d_{\text{model}})}$ is a matrix of weights and $\mathbf{b} \in \mathbf{R}^{d_{model}}$ is a vector of biases. Importantly, $\mathbf{W}$ and $\mathbf{b}$ are indexed with $_{\omega}$ to denote weight-sharing (within each variable) when the layer is time-distributed.

@dauphin2017language introduced GLUs. Their intuition is to train two versions of a dense layer in the same data, but one of them having a sigmoid activation (which outputs values between zero and one), then multiplying each hidden unit.

The result could be zero or very close to zero through the Hadamard multipliciation, which in practice means that the network would not be affected by that data (ie, the data $\gamma$ would be gated out). The first term, with the sigmoid, is the gate that determines what percentage of the linear layer passes through.

According to @lim2021temporal, GLUs:

* *"... reduce the vanishing gradient problem for deep architectures by providing a linear path for gradients while retaining non-linear capabilities"* and
* *"... provide flexibility to suppress any parts of the architecture that are not required for a given dataset"*

The GLU is a key part of the Gated Residual Network, described in @sec-GRN.

In [ ]:
#| code-fold: hide

class GatedLinearUnit(keras.Layer):
    def __init__(
        self,
        d_model:int=16, # Embedding size, $d_\text{model}$
        dropout_rate:float|None=None, # Dropout rate
        use_time_distributed:bool=True, # Apply the GLU across all time steps?
        activation:str|callable=None, # Activation function
        **kwargs
    ):
        "Gated Linear Unit dynamically gates input data"
        super(GatedLinearUnit, self).__init__(**kwargs)
        self.d_model = d_model
        self.dropout_rate = dropout_rate
        self.use_time_distributed = use_time_distributed
        self.activation = activation

    def build(self, input_shape):
        super(GatedLinearUnit, self).build(input_shape)
        self.dropout = layers.Dropout(self.dropout_rate) if self.dropout_rate is not None else None
        self.activation_layer = layers.Dense(self.d_model, activation=self.activation)
        self.gate_layer = layers.Dense(self.d_model, activation='sigmoid')
        self.multiply = layers.Multiply()

        if self.use_time_distributed:
            self.activation_layer = layers.TimeDistributed(self.activation_layer)
            self.gate_layer = layers.TimeDistributed(self.gate_layer)

    def call(
        self, 
        inputs, 
        training=None
    ):
        """List of outputs with shape: [
            (batch size, ..., d_model),
            (batch size, ..., d_model)
        ]"""
        if self.dropout is not None and training:
            inputs = self.dropout(inputs)

        activation_output = self.activation_layer(inputs)
        gate_output = self.gate_layer(inputs)
        return self.multiply([activation_output, gate_output]), gate_output

    def get_config(self):
        config = super(GatingLayer, self).get_config()
        config.update({
            'd_model': self.d_model,
            'dropout_rate': self.dropout_rate,
            'use_time_distributed': self.use_time_distributed,
            'activation': self.activation
        })
        return config

In [ ]:
#| output: asis
#| echo: false

show_doc(GatedLinearUnit.__init__, title_level=3)

In [ ]:
#| output: asis
#| echo: false

show_doc(GatedLinearUnit.call, title_level=3)

In [ ]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 100
d_model = 16

# input dimensions: batches / timesteps / features
x = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# output dimensions: batches / timesteps / d_model
print("GLU output shape: (batch size / num time steps / embedding dim)", GatedLinearUnit(d_model=16)(x)[0].shape)
print("GLU gate shape: (batch size / num time steps / embedding dim)", GatedLinearUnit(d_model=16)(x)[1].shape)

In [ ]:
#| code-fold: show

# output dimensions: batches / num time steps / num variables / d_model

glu = GatedLinearUnit()
gated_linear, gate = glu(xi_hist[:10, Ellipsis])
print("GLU input shape: ", xi_hist[:10, Ellipsis].shape)
print("GLU output shape: ", gated_linear.shape)
print("GLU gate shape:", gate.shape)

## Gated residual network (GRN) {#sec-GRN}

$$
\text{GRN}_{\omega}(a, c) = \text{LayerNorm}(a + \text{GLU}_{\omega}(W_{1, \omega} \text{ELU}(W_{2, \omega} a + b_{2, \omega} + W_{3, \omega} c) + b_{1, w}))
$$ {#eq-GRN}

* Breaking down $\text{GRN}_{\omega}(a, c)$:
    * *1st step*: $\eta_{2} = \text{ELU}(W_{2, \omega} a + b_{2, \omega} + W_{3, \omega} c)$ (where the additional context $c$ might be zero) as in @eq-dense but adapted for the added context if any and with $\text{ELU}(\cdot)$ as the activation function,
    * *2nd step*: $\eta_{1} = W_{1, \omega} \eta_{2} + b_{1, w}$ as in @eq-dense,
    * *3rd step*: $\text{LayerNorm}(a + \text{GLU}_{\omega}(\eta_{1}))$ as in @eq-skip and @eq-GLU
* $\text{ELU}(\cdot)$ is the Exponential Linear Unit activation function (@clevert2015fast)
    * Unlike ReLUs, ELUs allow for negative values, which pushes unit activations closer to zero at a lower computation complexity, and producing more accurate results
* The GRN is a key building block of the TFT
    * Helps keep information only from relevant input variables
    * Also keeps the model as simple as possible by only applying non-linearities when relevant

Note that the GRN can take all types of time series inputs, ie continuous historical, categorical historical and categorical future, but not categorical static data.

In [ ]:
#| code-fold: hide

class GatedResidualNetwork(keras.layers.Layer):
    def __init__(
        self, 
        d_model:int=16, # Embedding size, $d_\text{model}$
        output_size=None, 
        dropout_rate=None, 
        use_time_distributed=True, 
        **kwargs
    ):
        "Gated residual network"
        super(GatedResidualNetwork, self).__init__(**kwargs)
        self.d_model = d_model
        self.output_size = output_size if output_size is not None else d_model
        self.dropout_rate = dropout_rate
        self.use_time_distributed = use_time_distributed

    def build(self, input_shape):
        super(GatedResidualNetwork, self).build(input_shape)
        self.dense = layers.Dense(self.output_size)
        self.hidden_dense = layers.Dense(self.d_model)
        self.hidden_activation = layers.Activation('elu')
        self.context_dense = layers.Dense(self.d_model, use_bias=False)
        self.gating_layer = GatedLinearUnit(
            d_model=self.output_size, 
            dropout_rate=self.dropout_rate, 
            use_time_distributed=self.use_time_distributed, 
            activation=None)
        self.add = layers.Add()
        self.norm = layers.LayerNormalization()

        if self.use_time_distributed:
            self.dense = layers.TimeDistributed(self.dense)
            self.hidden_dense = layers.TimeDistributed(self.hidden_dense)
            self.context_dense = layers.TimeDistributed(self.context_dense)

    def call(self, inputs, additional_context=None, training=None):
        # Setup skip connection
        skip = self.dense(inputs) if self.output_size != self.d_model else inputs
        
        # 1st step: eta2
        hidden = self.hidden_dense(inputs)

        # Context handling
        if additional_context is not None:
            hidden += self.context_dense(additional_context)

        hidden = self.hidden_activation(hidden)

        # 2nd step: eta1 and 3rd step
        gating_layer, gate = self.gating_layer(hidden)
        
        # Final step
        GRN = self.add([skip, gating_layer])
        GRN = self.norm(GRN)

        return GRN, gate

    def get_config(self):
        config = super(GatedResidualNetwork, self).get_config()
        config.update({
            'd_model': self.d_model,
            'output_size': self.output_size,
            'dropout_rate': self.dropout_rate,
            'use_time_distributed': self.use_time_distributed
        })
        return config


In [ ]:
#| output: asis
#| echo: false

show_doc(GatedResidualNetwork.__init__, title_level=3)

In [ ]:
#| output: asis
#| echo: false

show_doc(GatedResidualNetwork.call, title_level=3)

In [ ]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 16
d_model = 16

# input dimensions: batches / timesteps / features
x = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

grn = GatedResidualNetwork(d_model=d_model)

output, gate = grn(x)

# output dimensions: batches / timesteps / d_model
print("GRN output shape: ", output.shape)
print("GRN gate shape: ", gate.shape)
#assert grn.shape == [batch_size, n_timesteps, output_size]

The example below uses transformed data $\xi_t^{(j)}$ for $j=0$ as an example.

In [ ]:
#| code-fold: show

grn = GatedResidualNetwork()
output, gate = grn(xi_hist[:,:,0,:])

# output dimensions: batches / timesteps / d_model (`output_size`)
output.shape, [xi_hist[:,:,0,:].shape[0], xi_hist[:,:,0,:].shape[1], d_model], gate.shape, [xi_hist[:,:,0,:].shape[0], xi_hist[:,:,0,:].shape[1], d_model]

## Variable selection networks

$$
\sum_{j=1}^{m_{\chi}} \upsilon_{\chi_t}^{(j)} \tilde{\xi}_t^{(j)},
$$ {#eq-VSN}

with $j$ indexing the input variable and $m$ being the number of features, $\upsilon_{\chi_t}^{(j)}$ standing for variable $j$'s selection weight, and $\tilde{\xi}_t^{(j)}$ defined as:

$$
\tilde{\xi}_t^{(j)} = \text{GRN}(\xi_t^{(j)}).
$$ {#eq-embed}

* In the paper, they are represented in the bottom right of Fig. 2
* Note there are separate variable selection networks for different input groups:
  * `static_variable_selection`
    * does not have static context as input, it already *is* the static information
  * `temporal_variable_selection`
    * used for both historical and known future inputs
    * includes static contexts
* Both of these functions take the result of the transformed data, ie embeddings for categorical variables and a linear layer for continuous variables
  * static variables are always categorical
  * temporal variables can be either categorical or continuous
  * in any case, following @lim2021temporal, the resulting transformation is expected to have the same dimension as `d_model`

In [ ]:
class StaticVariableSelection(keras.Layer):
    def __init__(
        self, 
        d_model:int=16, # Embedding size, $d_\text{model}$
        dropout_rate:float=0., 
        **kwargs
    ):
        "Static variable selection network"
        super(StaticVariableSelection, self).__init__(**kwargs)
        self.d_model = d_model
        self.dropout_rate = dropout_rate

        # Define GRNs for the transformed embeddings
        self.grns_transformed_embeddings = []  # This will be a list of GRN layers

        self.flat = layers.Flatten()
        self.softmax = layers.Activation('softmax')
        self.mult = layers.Multiply()

    def build(self, input_shape):
        super(StaticVariableSelection, self).build(input_shape)
        
        num_static = input_shape[2]

        # Define the GRN for the sparse weights
        self.grn_sparse_weights = GatedResidualNetwork(
            d_model=self.d_model,
            output_size=num_static,
            use_time_distributed=False
        )

        for i in range(num_static):
            # Create a GRN for each static variable
            self.grns_transformed_embeddings.append(
                GatedResidualNetwork(self.d_model, use_time_distributed=False)
            )

    def call(self, inputs):
        _, _, num_static, _ = inputs.shape # batch size / one time step (since it's static) / num static variables / d_model

        flattened = self.flat(inputs)

        # Compute sparse weights
        grn_outputs, _ = self.grn_sparse_weights(flattened)
        sparse_weights = self.softmax(grn_outputs)
        sparse_weights = keras.ops.expand_dims(sparse_weights, axis=-1)

        # Compute transformed embeddings
        transformed_embeddings = []
        for i in range(num_static):
            embed, _ = self.grns_transformed_embeddings[i](inputs[:, 0, i:i+1, :])
            transformed_embeddings.append(embed)
        transformed_embedding = keras.ops.concatenate(transformed_embeddings, axis=1)

        # Combine with sparse weights
        combined = self.mult([sparse_weights, transformed_embedding])
        static_vec = keras.ops.sum(combined, axis=1)

        return static_vec, sparse_weights

    def get_config(self):
        config = super(StaticVariableSelectionLayer, self).get_config()
        config.update({
            'd_model': self.d_model,
            'dropout_rate': self.dropout_rate
        })
        return config


In [ ]:
#| code-fold: show
#| output: asis

show_doc(StaticVariableSelection.__init__, title_level=3)

In [ ]:
#| code-fold: show
#| output: asis

show_doc(StaticVariableSelection.call, title_level=3)

In [ ]:
#| code-fold: show
#| warning: false

svars = StaticVariableSelection()
static_vars, static_selection_weights = svars(xi_stat)

print("xi_t shape: batch size / num time steps (=one) / number of categorical static variables / embedding size", xi_stat.shape)
print("Sum of the embeddings of selected variables shape: batch size / embedding size", static_vars.shape)
print("Selection weights shape: batch size / number of categorical static variables / one", static_selection_weights.shape)

In [ ]:
#| code-fold: hide

class TemporalVariableSelection(keras.Layer):
    def __init__(
        self, 
        d_model:int=16, # Embedding size, $d_\text{model}$
        dropout_rate:float=0., 
        **kwargs
    ):
        "Temporal variable selection"
        super(TemporalVariableSelection, self).__init__(**kwargs)
        self.d_model = d_model
        self.dropout_rate = dropout_rate

        self.mult = layers.Multiply()

    def build(self, input_shape):
        super(TemporalVariableSelection, self).build(input_shape)
        self.batch_size, self.time_steps, self.num_input_vars, self.d_model = input_shape[0]

        self.var_sel_weights = GatedResidualNetwork(
            d_model=self.d_model,
            output_size=self.num_input_vars,
            dropout_rate=self.dropout_rate,
            use_time_distributed=True,
        )
        self.softmax = layers.Activation('softmax')
    
        # Create a GRN for each temporal variable
        self.grns_transformed_embeddings = GatedResidualNetwork(
                d_model=self.d_model,
                dropout_rate=self.dropout_rate,
                use_time_distributed=True
        )
        # self.grns_transformed_embeddings = [
        #     GatedResidualNetwork(
        #         d_model=self.d_model,
        #         dropout_rate=self.dropout_rate,
        #         use_time_distributed=True
        #     ) for _ in range(self.num_input_vars)
        # ]


    def call(
        self, 
        inputs, # List of temporal embeddings, static context
        training=None
    ):
        temporal_embeddings, static_context = inputs

        flattened_embed = keras.ops.reshape(
            temporal_embeddings,
            [-1, self.time_steps, self.num_input_vars * self.d_model]
        )
        parallel_variables = keras.ops.reshape(
            temporal_embeddings, 
            [-1, self.time_steps, self.d_model]
        ) # tensor is shaped this way so that a GRN can be applied to each variable of each batch
        c_s = keras.ops.expand_dims(static_context, axis=1)

        # variable weights
        grn_outputs, _ = self.var_sel_weights(flattened_embed, c_s, training=training)
        variable_weights = self.softmax(grn_outputs)
        variable_weights = keras.ops.expand_dims(variable_weights, axis=2)

        # variable combination
        # transformed_embeddings = [
        #     grn_layer(temporal_embeddings[:, :, i, :], training=training)[0]
        #     for i, grn_layer in enumerate(self.grns_transformed_embeddings)
        # ]
        transformed_embeddings, _ = self.grns_transformed_embeddings(parallel_variables)
        transformed_embeddings = keras.ops.reshape(
            transformed_embeddings,
            [-1, self.time_steps, self.num_input_vars, self.d_model]
        )
        #transformed_embeddings = keras.ops.stack(transformed_embeddings, axis=2)
        temporal_vec = keras.ops.einsum('btij,btjk->btk', variable_weights, transformed_embeddings)
        return temporal_vec, keras.ops.squeeze(variable_weights, axis=2)

In [ ]:
#| code-fold: show
#| output: asis

show_doc(TemporalVariableSelection.__init__, title_level=3)

In [ ]:
#| code-fold: show
#| output: asis

show_doc(TemporalVariableSelection.call, title_level=3)

The main input to `temporal_variable_selection` are the transformed *temporal* variables $\xi_t^{(j)}$.

The selection of the temporal variables also requires the context from the static variables $c_s$, created below by passing `static_vars`, the output from the static variable selection unit (see @eq-static_var_sel), into a GRN.

In [ ]:
#| code-fold: show

# static context c_s
svars = StaticVariableSelection()
xi_stat_selected, _ = svars(xi_stat)
static_context_grn = GatedResidualNetwork(use_time_distributed=False)
c_s, _ = static_context_grn(xi_stat_selected)

# variable selection, historical data
tvars_hist = TemporalVariableSelection()
hist_sel, hist_weights = tvars_hist([xi_hist[:,:,:5,:], c_s]) # only the first five variables to keep it quick
print("Temporal variables feature shape: (batch size, num time steps, d_model)", hist_sel.shape)
print("Temporal variables weights shape: (batch size, num time steps, num variables (5))", hist_weights.shape)

# variable selection, future data
tvars_fut = TemporalVariableSelection()
fut_sel, fut_weights = tvars_fut([xi_fut, c_s])
print("Temporal variables feature shape: (batch size, num time steps, d_model)", fut_sel.shape)
print("Temporal variables weights shape: (batch size, num time steps, num variables)", fut_weights.shape)

## Sequence-to-sequence layer (LSTM)

This  the `TemporalFeatures` layer implements the following transformation:

$$ \text{LSTM} :
\tilde{\xi}_{t-k:t}, \tilde{\xi}_{t:\tau_{\text{max}}} \in \mathbb{R}^{k + \tau_{\text{max}} X d_{\text{model}}} \to \phi(t, n) \in \mathbb{R}^{k + \tau_{\text{max}} X d_{\text{model}}}, n \in [-k, \tau_{\text{max}}],
$$ {#eq-seqtoseq}

where the starting cell and hidden states of $\text{LSTM}_{t-k:t}$ are $c_c$ and $c_h$, each calculated as $c_p = GRN(\xi^{(j)}), p \in (c, h)$ and $j$ denoting static variables.

Finally, the `TemporalFeatures` layer compares the input data $\tilde{\xi}_{t-k:\tau_{\text{max}}}$ with the non-linear transformation $\phi(t, n)$, as follows:

$$
\tilde{\phi}(t, n) = \text{LayerNorm}(\tilde{\xi}_{t+n} + \text{GLU}_{\tilde{\phi}}(\phi(t, n))).
$$

In [ ]:
class TemporalFeatures(keras.Layer):
    def __init__(
        self, 
        d_model:int=16, # Embedding size, $d_\text{model}$
        dropout_rate:float=0., # Dropout rate
        **kwargs
    ):
        super(TemporalFeatures, self).__init__(**kwargs)
        self.d_model = d_model
        self.dropout_rate = dropout_rate

    def build(self, input_shape):
        super(TemporalFeatures, self).build(input_shape)
        self.hist_encoder = layers.LSTM(
            units=self.d_model,
            return_sequences=True,
            return_state=True,
            stateful=False,
            activation='tanh',
            recurrent_activation='sigmoid',
            recurrent_dropout=self.dropout_rate,
            unroll=False,
            use_bias=True
        )
        self.fut_decoder = layers.LSTM(
            units=self.d_model,
            return_sequences=True,
            return_state=False,
            stateful=False,
            activation='tanh',
            recurrent_activation='sigmoid',
            recurrent_dropout=self.dropout_rate,
            unroll=False,
            use_bias=True
        )
        self.lstm_glu = GatedLinearUnit(
            d_model=self.d_model, # Dimension of the GLU
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=True, # Apply the GLU across all time steps?
            activation=None # Activation function
        )
        self.add = layers.Add()
        self.l_norm = layers.LayerNormalization()

    def call(self, inputs, training=None):
        historical_features, future_features, c_h, c_c = inputs
        input_embeddings = keras.ops.concatenate(
            [historical_features, future_features],
            axis=1
        )

        history_lstm_encoder, state_h, state_c = self.hist_encoder(
            historical_features,
            initial_state=[
                c_h, # short-term state
                c_c  # long-term state
            ],
            training=training
        )

        future_lstm_decoder = self.fut_decoder(
            future_features,
            initial_state=[
                state_h, # short-term state
                state_c  # long-term state
            ],
            training=training
        )
        print(history_lstm_encoder.shape, future_lstm_decoder.shape)
        # this step concatenates at the time dimension, ie
        # the time series of history internal states are now
        # concated in sequence with the series of future internal states
        # $\phi(t,n) \in \{\phi(t,-k), \dots, \phi(t, \tau_{\text{max}})\}$
        lstm_layer = keras.ops.concatenate([history_lstm_encoder, future_lstm_decoder], axis=1)
        
        # Apply gated skip connection
        lstm_layer, _ = self.lstm_glu(
            lstm_layer,
            training=training
        )
        outputs = self.add([lstm_layer, input_embeddings])
        outputs = self.l_norm(outputs)
        # it's the temporal feature layer that is fed into the Temporal Fusion Decoder
        # its dimensions are (batch size / num time steps historical + future / hidden size)

        return outputs

In [ ]:
temp_feat = TemporalFeatures()

To illustrate the use of the `TemporalFeatures` layer, we first need to calculate the static contexts used to initialise the historical features' $h$ and $c$ state, respectively $c_h$ and $c_c$.

In [ ]:
# #| code-fold: show

# d_model=16
# dropout_rate=0.
# svars = StaticVariableSelection(
#     d_model=d_model,
#     dropout_rate=dropout_rate,
#     name="static_variable_selection"
# )
# c_s_grn = GatedResidualNetwork(
#     d_model=d_model,
#     dropout_rate=dropout_rate,
#     use_time_distributed=False,
#     name="static_context"
# )
# c_h_grn = GatedResidualNetwork(
#     d_model=d_model,
#     dropout_rate=dropout_rate,
#     use_time_distributed=False,
#     name="static_context"
# )
# c_c_grn = GatedResidualNetwork(
#     d_model=d_model,
#     dropout_rate=dropout_rate,
#     use_time_distributed=False,
#     name="static_context"
# )
# stat_vars, _ = svars(xi_stat)
# c_s, _ = c_s_grn(stat_vars)
# c_h, _ = c_h_grn(stat_vars)
# c_c, _ = c_c_grn(stat_vars)

# print("stat_vars shape: (batch size / d_model)", stat_vars.shape)
# print("c_s shape: (batch size / d_model)", c_s.shape)
# print("c_h shape: (batch size / d_model)", c_h.shape)
# print("c_c shape: (batch size / d_model)", c_c.shape)

Now these variables can be used as inputs for the temporal sequence encoding.

In [ ]:
# tvars_hist = TemporalVariableSelection(
#     d_model=d_model,
#     dropout_rate=dropout_rate,
#     name="historical_variable_selection"
# )
# tvars_fut = TemporalVariableSelection(
#     d_model=d_model,
#     dropout_rate=dropout_rate,
#     name="future_variable_selection"
# )
# temp_feat = TemporalFeatures()
# xi_hist_tilde, _ = tvars_hist([xi_hist, c_s])
# xi_fut_tilde, _ = tvars_fut([xi_fut, c_s])

# xi_hist_tilde.shape, xi_fut_tilde.shape

# temporal_features = temp_feat([xi_hist_tilde, xi_fut_tilde, c_h, c_c])
# print("temporal_features shape: (batch size / num time steps (hist + fut) / d_model)", temporal_features.shape)

In [ ]:
#| output: asis
#| echo: false

show_doc(TemporalFeatures.__init__, title_level=3)

In [ ]:
#| output: asis
#| echo: false

show_doc(TemporalFeatures.call, title_level=3)

The graph below plots the temporal futures resulting from the function above, using the first batch of data as an example. Each curve is the time series of one element of the embedding vector, which in turn contains all the relevant information from the time-varying inputs, both categorical and continuous, after being filtered by the network. The black vertical line marks the point at which the temporal futures is relying on future known information.

Note that even with these randomly initiated LSTM layers, it is already possible to see the obvious fact that the information content from historical input (left to the vertical line) is different compared to the known future data (ie, information from the dates; to the right of the vertical line).

Still, the future part has *some* information, which might be useful in nowcasting or predicting inflation farther out.

In [ ]:
# ax = pd.DataFrame(temporal_features[0,:,:]).plot(legend=False)
# ax.axvline(x=xi_hist_tilde.shape[1], color="black")

## Partial TFT model

Let's check out a partial TFT model, with only the building blocks described above.

In [ ]:
class PartialTFT(keras.Model):
    def __init__(
        self,
        # quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
        d_model:int=16, # Embedding size, $d_\text{model}$
        output_size:int=1, # How many periods to nowcast/forecast?
        n_head:int=4,
        dropout_rate:float=0.1,
        random_state:int=1985,
        **kwargs
    ):
        super(PartialTFT, self).__init__(**kwargs)
        # self.quantiles = quantiles
        self.d_model = d_model
        self.output_size = output_size
        self.n_head = n_head
        self.dropout_rate = dropout_rate
        self.random_state = random_state

        keras.utils.set_random_seed(self.random_state)

    def build(self, input_shape):
        super(PartialTFT, self).build(input_shape)
        
        self.input_layer = InputTFT(
            d_model=self.d_model,
            name="input"
        )
        self.svars = StaticVariableSelection(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            name="static_variable_selection"
        )
        self.tvars_hist = TemporalVariableSelection(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            name="historical_variable_selection"
        )
        self.tvars_fut = TemporalVariableSelection(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            name="future_variable_selection"
        )
        self.static_context_s_grn = GatedResidualNetwork(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=False,
            name="static_context"
        )
        self.static_context_h_grn = GatedResidualNetwork(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=False,
            name="static_context"
        )
        self.static_context_c_grn = GatedResidualNetwork(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=False,
            name="static_context"
        )
        self.temporal_features = TemporalFeatures(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            name="LSTM_encoder"
        )
        self.add = layers.Add()
        self.l_norm = layers.LayerNormalization()
        self.flat = layers.Flatten(name="flatten")

        output_len = self.output_size # * len(self.quantiles)
        # self.output_layer = layers.TimeDistributed(
        #     layers.Dense(output_len),
        #     name="output"
        # )
        self.output_layer = layers.Dense(output_len, name="output")
    
    def call(
        self,
        inputs
    ):
        "Creates the model architecture"
        
        # embedding the inputs
        cont_hist, cat_hist, cat_fut, cat_stat = inputs
        if len(cat_stat.shape) == 2:
            cat_stat = keras.ops.expand_dims(cat_stat, axis=-1)
            
        xi_hist, xi_stat, xi_fut = self.input_layer([cont_hist, cat_hist, cat_stat, cat_fut])

        # selecing the static covariates
        static_selected_vars, static_selection_weights = self.svars(xi_stat)

        # create context vectors from static data
        c_s, _ = self.static_context_s_grn(static_selected_vars) # for variable selection
        c_h, _ = self.static_context_h_grn(static_selected_vars) # for LSTM state h
        c_c, _ = self.static_context_c_grn(static_selected_vars) # for LSTM state c

        # temporal variable selection
        #print("starting hist tvars, with inputs shaped: ", xi_hist.shape, c_s.shape)
        hist_selected_vars, hist_selection_weights = self.tvars_hist([xi_hist, c_s])
        #print("starting fut tvars, with inputs shaped: ", xi_fut.shape, c_s.shape)
        fut_selected_vars, fut_selection_weights = self.tvars_fut([xi_fut, c_s])
        input_embeddings = keras.ops.concatenate(
            [hist_selected_vars, fut_selected_vars],
            axis=1
        )

        #print("starting LSTM encoding, with inputs shaped: ", hist_selected_vars.shape, fut_selected_vars, c_h.shape, c_c.shape)
        output = self.temporal_features(
            [hist_selected_vars, fut_selected_vars, c_h, c_c]
        )
        
        # flatten the outputs (similar to a "bag of future encodings")
        # otherwise there is an output per future known day
        output = self.flat(output[:,output.shape[1]:,:])
        output = self.output_layer(output)
        #print("forward pass complete, shape: ", output.shape)
        # define the outputs
        return output

In [ ]:
#| code-fold: show

tft = PartialTFT(
    d_model=6,
    output_size=train_data[1].shape[-1], 
    dropout_rate=0.2,
    name="partial_tft"
)
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = keras.losses.MeanSquaredError()
tft.compile(optimizer=optimizer, loss=loss_fn)

In [ ]:
hist = tft.fit(
    x=train_data[0], 
    y=train_data[1], 
    validation_data=valid_data,
    batch_size=100, 
    epochs=50,
)

In [ ]:
# train_dataloader = DataLoader(data, batch_size=5, shuffle=True)
tft.summary()

In [ ]:
val_dataloader = DataLoader(data, batch_size=1, shuffle=True)
x, y = next(iter(val_dataloader))
tft.predict(x)

In [ ]:
tft.save("model.keras")

## Including speech data
For now, simply merge the speech embeddings with the daily DataFrame

In [ ]:
speech_embeddings = pd.read_csv("../data/speech_embeddings.csv", index_col="date")
speech_embeddings.index = pd.to_datetime(speech_embeddings.index)
df_input_embed = df_input_scl.join(speech_embeddings, on="index").fillna(0)
df_input_embed.head()

In [ ]:
data = NowcastingData(
    df_daily_input=df_input_embed, 
    df_target=df_target_1m_pct,
    max_context_length_days=365,
    delta_month=range(12)
)

tft_speech = PartialTFT(
    d_model=8,
    output_size=y.shape[-1], 
    dropout_rate=0.2,
    name="partial_tft"
)

train_dataloader = DataLoader(data, batch_size=32, shuffle=True)
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = keras.losses.MeanSquaredError()
tft_speech.compile(optimizer=optimizer, loss="mse")

In [ ]:
batch = next(iter(train_dataloader))
history = tft_speech.fit(batch[0], batch[1], epochs=50)

In [ ]:
tft_speech.save("model_speech.keras")

## Static enrichment

This step is responsible for adding static information on the country for which inflation is being nowcasted to the temporal features.

This is achieved by a GRN layer as follows:

$$
\theta(t, n) = \text{GRN}_{\theta}(\tilde{\phi}(t, n), c_e)
$$

In [ ]:
def static_enrichment(
    temporal_features:keras.KerasTensor, # $\tilde{\phi}(t, n)$, temporal features of dimension (batch size / num historical + future time steps / embedding size)
    static_context:keras.KerasTensor, # $c_e$, static context enrichment vector of dimensions (batch size / num static vars / embedding size)
    d_model:int, # Embedding size, $d_\text{model}$
    dropout_rate:float=0. # Dropout rate
):
    "Static enrichment"
    enriched, _ = gated_residual_network(
        temporal_features,
        d_model=d_model,
        dropout_rate=dropout_rate,
        use_time_distributed=True,
        additional_context=static_context
    )
    return enriched

In [ ]:
show_doc(static_enrichment, title_level=3)

The code below first creates an static context enrichment vector:

In [ ]:
#| code-fold: show

static_context_enrichment, _ = gated_residual_network( # c_3
    x=xi_stat, # Network inputs
    d_model=d_model, # Dimension of the GRN
    output_size=d_model, # Size of output layer (if None, same as `d_model`)
    dropout_rate=0., # Dropout rate
    use_time_distributed=False, # Apply the GRN across all time steps?
)
static_context_enrichment.shape

... and now it is used with the temporal features that are the output of `seq_to_seq` for the static enrichment.

In [ ]:
enriched_temporal_features = static_enrichment(
    temporal_features=temporal_features,
    static_context=static_context_enrichment,
    d_model=16
)
temporal_features.shape, enriched_temporal_features.shape

## Attention components

* Attention mechanisms use relationships between keys $K \in \mathbf{R}^{N \times d_{attention}}$ and queries $Q \in \mathbf{R}^{N \times d_{attention}}$ to scale a vector of values $V \in \mathbf{R}^{N \times d_V}$: $\text{Attention}(Q, K, V) = A(Q, K) V$
    * $N$ is the number of timesteps going into the attention layer (the number of lags $k$ plus the number of periods to be forecasted $\tau_{\text{max}}$)
    * $A(\cdot)$ is a normalisation function
        * After @vaswani2017attention, the canonical choice for $A(\cdot)$ is the scaled dot-product: $A(Q, K) = \text{Softmax}(\frac{Q K^{T}}{\sqrt{d_{attention}}} )$
    
* The TFT uses a modified attention head to enhance the explainability of the model
* Specifically, the transformer block (multi-head attention) is modified to:
    * share values in each head, and
    * employ additive aggregation of all heads
* More formally, compare the interpretable multi-head attention (used in this paper) with the canonical multi-head attention:
    * $\text{InterpretableMultiHead}(Q, K, V) = \tilde{H} W_{H}$, with:
        * $\begin{aligned}\tilde{H} &= \tilde{A}(Q, K) V W_V \\
        &= \{\frac{1}{m_H} \sum^{m_{H}}_{h=1} A(Q W^{(h)}_Q, K W^{(h)}_K) \} V W_V \\
        &= \frac{1}{m_H} \sum^{m_{H}}_{h=1} \text{Attention}(Q W^{(h)}_Q, K W^{(h)}_K, V W_V)
        \end{aligned}$
    * $\text{MultiHead}(Q, K, V) = [H_1, \dots, H_{m_H}] W_H$, with:
        * $H_h = \text{Attention}(Q W^{(h)}_Q, K W^{(h)}_K, V W_V^{(h)}) $

## Decoder mask for self-attention layer

In [ ]:
def get_decoder_mask(
    self_attention_inputs # Inputs to the self-attention layer
):
    "Determines shape of decoder mask"
    len_s = keras.ops.shape(self_attention_inputs)[1] # length of inputs
    bs = keras.ops.shape(self_attention_inputs)[0] # batch shape
    mask = keras.ops.cumsum(keras.ops.eye(len_s), axis=0)

    ### warning: I had to manually implement some batch-wise shape here 
    ### because the new keras `eye` function does not have a batch_size arg.
    ### inspired by: https://github.com/tensorflow/tensorflow/blob/v2.14.0/tensorflow/python/ops/linalg_ops_impl.py#L30
    ### <hack>
    mask = keras.ops.expand_dims(mask, axis=0)    
    mask = keras.ops.tile(mask, (bs, 1, 1))
    ### </hack>

    return mask


In [ ]:
#| output: asis
#| echo: false

show_doc(get_decoder_mask, title_level=3)

#### Example usage

In [ ]:
#| code-fold: show

dec = get_decoder_mask(grn)

assert dec.shape == (grn.shape[0], grn.shape[1], grn.shape[1])

Note that it produces an upper-triangular matrix of ones:

In [ ]:
#| code-fold: show

dec[0]

#### Example usage, real data

In [ ]:
mask = get_decoder_mask(temporal_features)

print("Mask shape: (batch size / num time seos / num time steps)", mask.shape)

mask[0]

## Scaled dot product attention layer

* This is the same as Eq. (1) of @vaswani2017attention 
    * except that in this case the dimension of the value vector is the same $d_{\text{attn}} = d_{\text{model}} / m_{\text{Heads}}$ as for the query and key vectors
* As discussed in the paper, additive attention outperforms dot product attention for larger $d_{\text{model}}$ values, so the attention is scaled back to smaller values

In [ ]:
class ScaledDotProductAttention(keras.Layer):
    def __init__(
        self,
        attention_dropout:float=0.0 # Will be ignored if `training=False`
    ):
        super().__init__()
        self.dropout = keras.layers.Dropout(rate=attention_dropout)
        self.activation = keras.layers.Activation('softmax')

    def __call__(
        self,
        q, # Queries, tensor of shape (?, time, D_model)
        k, # Keys, tensor of shape (?, time, D_model)
        v, # Values, tensor of shape (?, time, D_model)
        mask, # Masking if required (sets Softmax to very large value), tensor of shape (?, time, time)
        training=None, # Whether the layer is being trained or used in inference
    ):
        # returns Tuple (layer outputs, attention weights)
        scale = keras.ops.sqrt(keras.ops.cast(keras.ops.shape(k)[-1], dtype='float32'))
        attention = layers.Dot(axes=(2, 2))([q, k]) / scale
        #attention = keras.ops.einsum("bij,bjk->bik", q, keras.ops.transpose(k, axes=(0, 2, 1))) / scale
        if mask is not None:
            mmask = keras.layers.Lambda(lambda x: (-1e9) * (1. - keras.ops.cast(x, 'float32')))(mask)
            attention = keras.layers.Add()([attention, mmask])
        attention = self.activation(attention)
        if training:
            attention = self.dropout(attention)
        output = layers.Dot(axes=(2, 1))([attention, v])
        #output = keras.ops.einsum("btt,btd->bt", attention, v)
        return output, attention

In [ ]:
#| output: asis
#| echo: false

show_doc(ScaledDotProductAttention, title_level=3)

#### Example usage
Below is an example of how the `ScaledDotProductAttention` layer works:

In [ ]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 13

# input dimensions: batches / timesteps / features
x_btf = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# using the same vector for q, k and v just to simplify
q=keras.ops.cast(x_btf, 'float32')
k=keras.ops.cast(x_btf, 'float32')
v=keras.ops.cast(x_btf, 'float32')

Testing without masking:

In [ ]:
#| code-fold: show

output, attention = ScaledDotProductAttention()(q=q, k=k, v=v, mask=None)
output.shape, attention.shape

... and with masking:

In [ ]:
#| code-fold: show

output, attention = ScaledDotProductAttention()(q=q, k=k, v=v, mask=get_decoder_mask(q))
output[0], attention[0] # both have shape (batch_size, n_timesteps)

#### Example usage, real data

In [ ]:
#| code-fold: show

mask = get_decoder_mask(enriched_temporal_features)

output, attention = ScaledDotProductAttention()(
    q=enriched_temporal_features,
    k=enriched_temporal_features,
    v=enriched_temporal_features,
    mask=mask
)

print("attention shape: (batch size / num time steps / embedding size)", output.shape)
print("attention shape: (batch size / num time steps / num time steps)", attention.shape)

The attention weights for the future periods are shown below.

In [ ]:
attention[0,-xi_fut.shape[1]:]

## Softmax

A small detour to illustrate the softmax function. 

The $i^{\text{th}}$ element of $\text{Softmax}(x)$, with $x \in \mathbf{R}^K$ is:

$$
\text{Softmax}(x)_i = \frac{e^{x_i}}{\sum_{j=1}^K e^{x_j}}
$$

For example, see the values below for an input vector $x$ ($K=5$ in this example):

In [ ]:
#| code-fold: show

x = np.array([-np.Inf, -1., 0., 1., 3.])
keras.layers.Activation('softmax')(x)
print("x = ", x)
print("exp(x) = ", np.exp(x))
print("denominator (sum of exp(x_j), j=1,...,K) = ", sum(np.exp(x)))
print("softmax(x) = ", np.exp(x) / sum(np.exp(x)))
print("sum of softmax(x)_j, j=1,...,K = ", sum(np.exp(x) / sum(np.exp(x))))

As can be seen above, the softmax function really makes the largest numbers stand out from the rest.

Note also that $-\infty$ results in 0.

## Interpretable Multi-head attention

* When values are shared in each head and then are aggregated additively, each head can still learn different temporal patterns (from their own unique queries and keys), but with the same input values.
    * In other words, they can be interpreted as an ensemble over the attention weights
    * the paper doesn't mention this explicitly, but the ensemble is equally-weighted - maybe there is some performance to be gained by having some way to weight the different attention heads 🤔, such as having a linear layer combining them... will explore in the future

In [ ]:
class InterpretableMultiHeadAttention(keras.Layer):
    def __init__(
        self,
        n_head:int,
        d_model:int, # Embedding size, $d_\text{model}$
        dropout_rate:float, # Will be ignored if `training=False`
    ):
        super().__init__()
        self.n_head = n_head
        self.d_k = self.d_v = d_model // n_head # the original model divides by number of heads
        self.dropout_rate = dropout_rate

        # using the same value layer facilitates interpretability
        vs_layer = keras.layers.Dense(self.d_v, use_bias=False, name="Shared value")

        # creates list of queries, keys and values across heads
        self.qs_layers = self._build_layers(self.d_k, n_head)
        self.ks_layers = self._build_layers(self.d_k, n_head)
        self.vs_layers = [vs_layer for _ in range(n_head)]

        self.attention = ScaledDotProductAttention()
        self.w_o = keras.layers.Dense(self.d_v, use_bias=False, name="W_v") # W_v in Eqs. (14)-(16), output weight matrix to project internal state to the original TFT

    def call(
        self,
        q, # Queries, tensor of shape (?, time, D_model)
        k, # Keys, tensor of shape (?, time, D_model)
        v, # Values, tensor of shape (?, time, D_model)
        mask=None, # Masking if required (sets Softmax to very large value), tensor of shape (?, time, time)
        training=None
    ):
        heads = []
        attns = []
        for i in range(self.n_head):
            qs = self.qs_layers[i](q)
            ks = self.ks_layers[i](q)
            vs = self.vs_layers[i](v)
           
            head, attn = self.attention(qs, ks, vs, mask, training=training)
            if training:
                head = keras.layers.Dropout(self.dropout_rate)(head)
            heads.append(head)
            attns.append(attn)
        head = keras.ops.stack(heads) if self.n_head > 1 else heads[0]

        outputs = keras.ops.mean(heads, axis=0) if self.n_head > 1 else head # H_tilde
        outputs = self.w_o(outputs)
        if training:
            outputs = keras.layers.Dropout(self.dropout_rate)(outputs)

        return outputs, attn

    def _build_layers(self, d:int, n_head:int):
        return [keras.layers.Dense(d) for _ in range(n_head)]

In [ ]:
#| output: asis
#| echo: false

show_doc(InterpretableMultiHeadAttention, title_level=3)

#### Example usage

In [ ]:
#| code-fold: show

imha = InterpretableMultiHeadAttention(n_head=8, d_model=16, dropout_rate=0.)

In [ ]:
#| code-fold: show

grn.shape # B, T, F

In [ ]:
#| code-fold: show

mask = get_decoder_mask(grn)
mask.shape # shape (batch size / num time steps / num time steps)

In [ ]:
#| code-fold: show

output, attn = imha(grn, grn, grn, mask)
output.shape, attn.shape

#### Exampel usage, real data

In [ ]:
#| code-fold: show

mask = get_decoder_mask(enriched_temporal_features)

output, attention = InterpretableMultiHeadAttention(
    n_head=1,
    d_model=16,
    dropout_rate=0.
)(
    q=enriched_temporal_features,
    k=enriched_temporal_features,
    v=enriched_temporal_features,
    mask=mask
)

print("attention shape: (batch size / num time steps / embedding size)", output.shape)
print("attention shape: (batch size / num time steps / num time steps)", attention.shape)

## Putting it all together: TFT

In [ ]:
###

class PartialTFT(keras.Model):
    def __init__(
        self,
        d_model:int=16, # Embedding size, $d_\text{model}$
        output_size:int=1
    ):
        super().__init__()
        self.d_model = d_model
        self.output_size = output_size
        
        self.input_layer = InputTFT(
            d_model=self.d_model,
            name="Input"
        )
        
        self.flat = layers.Flatten()
        self.concat = layers.Concatenate(axis=-1)
        self.dense_hidden = layers.Dense(self.d_model)
        self.dense_output = layers.Dense(self.output_size)
    
    def call(
        self,
        input:list # List with [continuous historical, categorical historical, categorical static, categorical future data]
    ):
        cont_hist, cat_hist, cat_stat, cat_fut = input
        if len(cat_stat.shape) == 2:
            cat_stat = keras.ops.expand_dims(cat_stat, axis=-1)

        hist, cat_stat, fut = self.input_layer([cont_hist, cat_hist, cat_stat, cat_fut])

        hist = self.flat(hist)
        cat_stat = self.flat(cat_stat)
        fut = self.flat(fut)

        output = self.concat([hist, cat_stat, fut])

        output = self.dense_hidden(output)
        output = self.dense_output(output)
        return output

model = PartialTFT(output_size=batch_y.shape[-1])
model.compile(optimizer="rmsprop", loss="mse")
model.fit([batch_cont_hist, batch_cat_hist, batch_cat_stat, batch_cat_fut], batch_y, epochs=5)

In [ ]:
class TemporalFusionTransformer(keras.Model):
    def __init__(
        self,
        quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
        d_model:int=16, # Embedding size, $d_\text{model}$
        output_size:int=1, # How many periods to nowcast/forecast?
        n_head:int=4,
        dropout_rate:float=0.1,
        random_state:int=1985,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.quantiles = quantiles
        self.d_model = d_model
        self.output_size = output_size
        self.n_head = n_head
        self.dropout_rate = dropout_rate
        self.random_state = random_state

        keras.utils.set_random_seed(self.random_state)

        self.input_layer = InputTFT(
            d_model=self.d_model,
            output_size=self.output_size
        )
        self.self_attention = InterpretableMultiHeadAttention(
            n_head=self.n_head,
            d_model=self.d_model,
            dropout_rate=self.dropout_rate
        )

        output_len = self.output_size * len(self.quantiles)
        self.output_layer = layers.TimeDistributed(
            layers.Dense(output_len)
        )

    def call(
        self,
        inputs,
        training=None
    ):
        "Creates the model architecture"

        # embedding the inputs
        cont_hist, cat_hist, cat_stat, cat_fut = inputs
        if len(cat_stat.shape) == 2:
            cat_stat = keras.ops.expand_dims(cat_stat, axis=-1)
            
        hist, cat_stat, fut = self.input_layer([cont_hist, cat_hist, cat_stat, cat_fut])

        # selecing the static covariates
        static_selected_vars, static_selection_weights = static_variable_selection(
            xi_stat, 
            d_model=d_model, 
            dropout_rate=0.
        )

        # create context vectors from static data
        static_context_variable_selection, _ = gated_residual_network(
            static_selected_vars,
            self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=False
        )
        
        static_context_enrichment, _ = gated_residual_network(
            static_selected_vars,
            self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=False
        )

        static_context_state_h, _ = gated_residual_network(
            static_selected_vars,
            self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=False
        )

        static_context_state_c, _ = gated_residual_network(
            static_selected_vars,
            self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=False
        )

        # temporal variable selection
        temporal_hist_selected_vars, temporal_hist_selection_weights = temporal_variable_selection(
            embedding=xi_hist,
            context=static_context_variable_selection,
            d_model=d_model, 
            dropout_rate=0.
        )

        temporal_fut_selected_vars, temporal_fut_selection_weights = temporal_variable_selection(
            embedding=xi_fut,
            context=static_context_variable_selection,
            d_model=d_model, 
            dropout_rate=self.dropout_rate
        )

        # sequence-to-sequence encoding
        temporal_features = seq_to_seq(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            historical_features=temporal_hist_selected_vars,
            future_features=temporal_fut_selected_vars,
            static_context_state_h=static_context_state_h,
            static_context_state_c=static_context_state_c
        )

        # static enrichment
        expanded_context = keras.ops.expand_dims(
            static_context_enrichment,
            axis=1
        )

        enriched_temporal_features = static_enrichment(
            temporal_features=temporal_features,
            static_context=expanded_context,
            d_model=self.d_model
        )

        mask = get_decoder_mask(enriched_temporal_features)

        features, self.attn_ = self.self_attention(
            q=enriched_temporal_features,
            k=enriched_temporal_features,
            v=enriched_temporal_features,
            mask=mask,
            training=training
        )

        # compare pre-attention features with post
        features, _ = apply_gating_layer(
            features,
            self.d_model,
            dropout_rate=self.dropout_rate,
            activation=None
        )
        features = add_and_norm([features, enriched_temporal_features])

        # non-linear processing of self-attention output
        decoder, _ = gated_residual_network(
            features,
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=True
        )

        # final skip connection
        decoder, _ = apply_gating_layer(
            decoder, 
            self.d_model, 
            activation=None
        )
        transformer_layer = add_and_norm([decoder, temporal_features])

        # define the outputs
        return self.output_layer(
            transformer_layer[:,xi_hist.shape[1]:,:]
        )

In [ ]:
cont_hist, cat_hist, stat, fut, y = batch_breakdown(batch)

tft = TemporalFusionTransformer(output_size=3)
output = tft([cont_hist, cat_hist, stat, fut])

In [ ]:
#| code-fold: show

cont_hist, cat_hist, stat, fut, y = batch_breakdown(batch)

tft = TemporalFusionTransformer(output_size=3)
tft.compile(optimizer="rmsprop", loss="mse")
output = tft.fit([cont_hist, cat_hist, stat, fut], y)

In [ ]:
print(tft.summary())

___

In [ ]:
class TemporalFusionTransformer():
    def __init__(
        self,
        # Data params
        time_steps:int,
        input_size:int,
        output_size:int,
        category_counts:int,
        n_workers:int, # Number of multiprocessing workers

        # TFT params
        input_obs_loc,
        static_input_loc,
        known_regular_input_idx,
        known_categorical_input_idx,
        column_definition,

        # Network params
        quantile:list=[0.1, 0.5, 0.9], # List of quantiles the model should forecast
        d_model:int=30, # Embedding size, $d_\text{model}$
        dropout_rate:float=0.0, # Dropout ratio (between 0.0, inclusive, and less than 1.0)
        num_encoder_steps:int=4,
        num_stacks:int=4,
        num_heads:int=4,
        
        # Training params
        max_gradient_norm:float=1.0, # 
        learning_rate:float=0.001,
        minibatch_size:int=64,
        num_epochs:int=100,
        early_stopping_patience:int=5,
        use_gpu:bool=True
    ):
        self.time_steps = time_steps
        self.input_size = input_size
        self.output_size = output_size # Number of periods to be forecasted
        self.category_counts = category_counts
        self.n_workers = n_workers # Number of multiprocessing workers
        
        self.input_obs_loc = input_obs_loc
        self.static_input_loc = static_input_loc
        self.known_regular_input_idx = known_regular_input_idx
        self.known_categorical_input_idx = known_categorical_input_idx
        self.column_definition = column_definition

        self.quantile = quantile # List of quantiles the model should forecast
        self.d_model = d_model # Size of hidden layer
        self.dropout_rate = dropout_rate # Dropout ratio (between 0.0, inclusive, and less than 1.0)
        self.num_encoder_steps = num_encoder_steps
        self.num_stacks = num_stacks
        self.num_heads = num_heads
        
        self.max_gradient_norm = max_gradient_norm
        self.learning_rate = learning_rate
        self.minibatch_size = minibatch_size
        self.num_epochs = num_epochs
        self.early_stopping_patience = early_stopping_patience
        self.use_gpu = use_gpu

        self.model = self.build_model()

    def __get_tft_embeddings(
        self,
        all_inputs # Input tensor of dimensions (batch, time steps, num variables)
    ):
        # Transform raw inputs to embeddings
        # For continuous variables: linear transformation
        # For categorical variables: embeddings
        
        num_categorical_variables = len(self.category_counts)
        num_regular_variables = self.input_size - num_categorical_variables

        embedding_sizes = [
            self.d_model
            for i, size in enumerate(self.category_counts)
        ]

        embeddings = [
            keras.Sequential([
                layers.InputLayer([self.time_steps]),
                layers.Embedding(
                    self.category_counts[i],
                    embedding_sizes[i],
                    input_length=self.time_steps,
                    dtype='float32'
                )
            ])
            for i in range(num_categorical_variables)
        ]

        regular_inputs, categorical_inputs = \
            all_inputs[:, :, :num_regular_variables], \
            all_inputs[:, :, num_regular_variables:]

        embedded_inputs = [
            embeddings[i](categorical_inputs[Ellipsis, i])
            for i in range(num_categorical_variables)
        ]

        # static inputs
        if self._static_input_loc:
            st_inp_dense = [
                layers.Dense(self.d_model)(
                    regular_inputs[:, 0, i:i + 1]
                )
                for i in range(num_regular_variables)
                if i in self._static_input_loc
            ]
            st_inp_embed = [
                embedded_inputs[i][:, 0, :]
                for i in range(num_categorical_variables)
                if  i + num_regular_variables in self._static_input_loc
            ]
            static_inputs = st_inp_dense + st_inp_embed
        else:
            static_inputs = None

        # Targets
        past_inputs = keras.ops.stack([
            dense_layer(
                size=self.d_model,
                activation=None,
                use_time_distributed=True
            )(regular_inputs[Ellipsis, i:i + 1])
        ], axis=-1)

        # past inputs: observed but not known a priori
        wired_embeddings = [
            embeddings[i](categorical_inputs[:,:,i])
            for i in range(num_categorical_variables)
            if i not in self._known_categorical_input_idx \
                and i + num_regular_variables not in self._input_obs_loc    
        ]
        unknown_inputs = [
            dense_layer(
                size=self.d_model,
                activation=None,
                use_time_distributed=True
            )(regular_inputs[Ellipsis, i:i + 1])
            for i in range(regular_inputs.shape[-1])
            if i not in self._known_categorical_input_idx \
                and i + num_regular_variables not in self._input_obs_loc    
        ]
        if wired_embeddings + unknown_inputs:
            unknown_inputs = keras.ops.stack(wired_embeddings + unknown_inputs, axis=-1)
        else:
            unkown_inputs = None

        # a priori known inputs
        known_regular_inputs = [
            dense_layer(
                size=self.d_model,
                activation=None,
                use_time_distributed=True
            )(regular_inputs[Ellipsis, i:i + 1])
            for i in self._known_regular_input_idx
            if i not in self._static_input_loc
        ]
        known_categorical_inputs = [
            embedded_inputs[i]
            for i in self._known_categorical_input_idx
            if i + num_regular_variables not in self._static_input_loc
        ]
        known_combined_layer = keras.ops.stack(
            known_regular_inputs + known_categorical_inputs,
            axis=-1
        )

        return unknown_inputs, known_combined_layer, past_inputs, static_inputs

    def _build_base_graph(self):
        # Build the graph, defining the layers of the TFT
        

        ### <TFTInputs>
        all_inputs = layers.Input(
            shape=(self.time_steps, self.input_size) # Argument `shape` does not include batch size
        )
        unknown_inputs, known_combined_layer, past_inputs, static_inputs \
            = self.__get_tft_embeddings(all_inputs)
        ### </TFTInputs>

        # first we isolate the known future inputs and observed past inputs
        if unknown_inputs is not None:
            historical_inputs = keras.ops.concatenate([
                unknown_inputs[:, :self.num_encoder_steps, :],
                known_combined_layer[:, :self.num_encoder_steps, :],
                past_inputs[:, :self.num_encoder_steps, :]
            ], axis=1)
        else:
            historical_inputs = keras.ops.concatenate([
                known_combined_layer[:, :self.num_encoder_steps, :],
                past_inputs[:, :self.num_encoder_steps, :]
            ])
        
        # and then we isolate the known future inputs
        future_inputs = known_combined_layer[:, :self.num_encoder_steps, :]

        # static vars
        static_encoder, static_weights = static_variable_selection(static_inputs)

        # Static covariate encoders
        # These integrate static features into the network through encoding of context vectors
        # that condition the time-varying dynamics
        self.static_context_variable_selection = gated_residual_network( # c_s
            x=static_encoder, # Network inputs
            d_model=self.d_model, # Dimension of the GRN
            output_size=self.d_model, # Size of output layer (if None, same as `d_model`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all time steps?
        )
        self.static_context_enrichment = gated_residual_network( # c_3
            x=static_encoder, # Network inputs
            d_model=self.d_model, # Dimension of the GRN
            output_size=self.d_model, # Size of output layer (if None, same as `d_model`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all time steps?
        )
        self.static_context_state_h = gated_residual_network( # c_h
            x=static_encoder, # Network inputs
            d_model=self.d_model, # Dimension of the GRN
            output_size=self.d_model, # Size of output layer (if None, same as `d_model`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all time steps?
        )
        self.static_context_state_c = gated_residual_network( # c_c
            x=static_encoder, # Network inputs
            d_model=self.d_model, # Dimension of the GRN
            output_size=self.d_model, # Size of output layer (if None, same as `d_model`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all time steps?
        )

        historical_features, historical_flags, _ = temporal_variable_selection(
            embedding=historical_inputs,
            context=self.static_context_variable_selection,
            d_model=self.d_model,
            dropout_rate=self.dropout_rate)
        future_features, future_flags, _ = temporal_variable_selection(
            embedding=future_inputs,
            context=self.static_context_variable_selection,
            d_model=self.d_model,
            dropout_rate=self.dropout_rate)

        # Locality enhancement (Section 4.5.1 in paper) with seq-to-seq layer

        temporal_feature_layer = seq_to_seq(
            d_model=self.d_model,
            dropout_rate=self.dropout_rate,
            historical_features=historical_features,
            future_features=future_features,
            static_context_state_c=self.static_context_state_c,
            static_context_state_h=self.static_context_state_h
        )
        # Temporal Fusion Decoder (TFT, Purple box in Fig. 2)
        # contains three steps
        # TFT 1st step: Static enrichment
        #   - enhances the temporal features with static metadata (Eq. 18)
        
        expanded_static_context_c_e = keras.ops.expand_dims(
            self.static_context_enrichment,
            axis=1
        )
        enriched = gated_residual_network( # $\theta(t, n) = \text{GRN}_{\theta}(\tilde{\theta}(t, n), c_e)
            x=temporal_feature_layer, # from t-k to t+\tau_max
            d_model=self.d_model,
            output_size=self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=True,
            additional_context=expanded_static_context_c_e,
            return_gate=False
        )

        # TFT 2nd step: Temporal self-attention

        self_attention_layer = InterpretableMultiHeadAttention(
            n_head=self.num_heads,
            d_model=self.d_model,
            dropout=self.dropout_rate # Will be ignored if `training=False`
        )
        mask = get_decoder_mask(enriched)
        post_attn, self_attention = self_attention_layer( # $B(t) = \text{IMHA}(\Theta(t), \Theta(t), \Theta(t))$
            q=enriched,
            k=enriched,
            v=enriched,
            mask=mask
        )
        post_attn, _ = apply_gating_layer( # $\text{GLU}_{\delta}(\beta(t, n))$
            x=post_attn, # Input tensors (batch first)
            d_model=self.d_model, # Dimension of the GLU
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=True, # Apply the GLU across all time steps?
            activation=None # Activation function
        )

        # skip connection: decides how much of the attention layer is used
        post_attn = add_and_norm([post_attn, enriched]) # \delta(t, n) = \text{LayerNorm}(\theta(t, n) + $\text{GLU}_{\delta}(\beta(t, n)))$

        # TFT 3rd step: Position-wise feed-forward
        decoder = gated_residual_network(
            x=post_attn,
            d_model=self.d_model,
            output_size=self.d_model,
            dropout_rate=self.dropout_rate,
            use_time_distributed=True,
            additional_context=None,
            return_gate=False
        )

        # final skip connection
        decoder, _ = apply_gating_layer(
            x=decoder, # Input tensors (batch first)
            d_model=self.d_model, # Dimension of the GLU
            dropout_rate=self.dropout_rate, # Dropout rate
            activation=None # Activation function
        )
        # the temporal feature layer here is to help the model decide
        # how much to skip the Temporal Fusion Decoder altogether
        transformer_layer = add_and_norm([decoder, temporal_feature_layer])

        # the function also returns the attention components
        # for explainability analyses
        attention_components = {
            "temporal_attention_weights": self_attention,
            "variable_selection_weights_static_inputs": static_weights[Ellipsis, 0],
            "variable_selection_weights_past_inputs": historical_flags[Ellipsis, 0, :],
            "variable_selection_weights_future_inputs": future_flags[Ellipsis, 0, :]
        }

        return transformer_layer, all_inputs, attention_components

    def build_model(self):
        # Build model and define training losses

        transformer_layer, all_inputs, self._attention_components = self._build_base_graph()
        outputs = keras.layers.TimeDistributed(
            keras.layers.Dense(self.output_size * len(self.quantiles))
        )(transformer_layer[Ellipsis, self.num_encoder_steps:, :])
        model = keras.Model(inputs=all_inputs, outputs=outputs)

# Using the TFT model

## Data

In this example, we will use a simple inflation panel dataset.

In [ ]:
from gingado.utils import list_all_dataflows, load_SDMX_data

In [ ]:
# dflows = list_all_dataflows()
# dflows[dflows == 'BIS long consumer prices']

In [ ]:
df_cpi = load_SDMX_data(
    sources={'BIS': 'WS_LONG_CPI'},
    keys={'FREQ': 'M'},
    params={'startPeriod': 2002}
)

In [ ]:
df_cpi = df_cpi['2002-01-01':'2023-09-01']
cpi_cols = df_cpi.dropna(how='all', axis=1).columns

In [ ]:
# keep only one observation per country, retaining only the ones ending in 628 (in opposition to 771)
df_infl = df_cpi[[c for c in cpi_cols if '628' in c and c != 'BIS__WS_LONG_CPI_M__AE__628']].pct_change(periods=1).dropna(how='all')

Clean the titles from the metadata

In [ ]:
df_infl.columns = [c.split('__')[2] for c in df_infl.columns]

In [ ]:
df_infl.plot(legend=False)

In [ ]:
train_cutoff = '2013-01-01'
valid_cutoff = '2020-01-01'
df_infl_train, df_infl_valid, df_infl_test = df_infl[:train_cutoff], df_infl[train_cutoff:valid_cutoff], df_infl[valid_cutoff:]
df_infl_train = df_infl_train[:-1]
df_infl_valid = df_infl_valid[:-1]

In [ ]:
df_infl_valid

### Data preparation

This crucial step involves:
* measuring the mean and standard deviation of the inflation of each country in the training dataset
* using the values above to standardise the training, validation and testing datasets

In [ ]:
train_scale = {
    'mean': df_infl_train.mean(axis=0),
    'std': df_infl_train.std(axis=0)
}

In [ ]:
def normalise_df(df, scale=train_scale):
    norm_dfs = []
    for cty in df.columns.tolist():
        norm_dfs.append((df[cty] - scale['mean'][cty]) / scale['std'][cty])
    return pd.concat(norm_dfs, axis=1)

In [ ]:
def renormalise_df(df, scale=train_scale):
    renorm_dfs = []
    for cty in df.columns.tolist():
        renorm_dfs.append((df[cty] * scale['std'][cty]) + scale['mean'][cty])
    return pd.concat(renorm_dfs, axis=1)

In [ ]:
# the code below check that it is working:
# (renormalise_df(normalise_df(df_infl_train, train_scale)) - df_infl_train)

In [ ]:
df_infl_train_n = normalise_df(df_infl_train)
df_infl_valid_n = normalise_df(df_infl_valid)
df_infl_test_n = normalise_df(df_infl_test)

In [ ]:
df_infl_valid_n.plot(legend=False)

## A simple dense layer

This first model is autoregressive: it takes in $p$ lags of an inflation series $\pi_i$ (in other words, $\pi_{i, t-p}, ..., \pi_{i, t-1}$) to predict the period $\pi_{i,t}$.

Note that the model is very simple:
* each country's inflation series is only predicted by its past values
* the fully connected linear layer learns to pick up any meaningful non-linear interactions between lags, but there is no intrinsic meaning in the order of the lags
* this network will always take in as input a $p$-sized vector of lagged data

### Data formatting

Let's create a simple function that will take a data frame and return a (input, output) tuple for the model.

In [ ]:
def dense_layer_io(
    df, # Pandas DataFrame with the data
    idx:[Int], # DataFrame indedx
    col:[str], # Column name
    p:Int # Number of lags
):
    # Return an (input, output) tuple for a dense_layer model
    
    assert len(idx) == len(col)

    batch_X, batch_y = [], []
    for i in range(len(idx)):
        X = df.iloc[idx[i]:(idx[i]+p)][[col[i]]].values.reshape(-1)
        if np.isnan(X).any():
            continue
        batch_X.append(X)
        batch_y.append(df.iloc[idx[i]+p][[col[i]]].values)
    
    return np.vstack(batch_X), np.vstack(batch_y)

    #return df.iloc[idx:(idx+p)][[col]].values.reshape(-1), df.iloc[idx+p+1][[col]].values

In [ ]:
dense_layer_io(df=df_infl_train, idx=[3, 4, 5], col=['BR', 'BR', 'BR'], p=4)

In [ ]:
def random_select(df, p, n_samples):
    max_index = len(df) - p
    rand_idx, rand_col = [], []
    for _ in range(n_samples):
        rand_idx.append(random.randint(0, max_index-2)) # instead of -1 only, we also need to account for the target value
        rand_col.append(random.choice(df.columns))
    return rand_idx, rand_col

In [ ]:
n_samples = 1000 # with replacement

In [ ]:
p = 24

idx, col = random_select(df_infl_train_n, p=p, n_samples=n_samples)
train_samples = dense_layer_io(df=df_infl_train_n, idx=idx, col=col, p=p)

idx, col = random_select(df_infl_valid_n, p=p, n_samples=n_samples)
valid_samples = dense_layer_io(df=df_infl_valid_n, idx=idx, col=col, p=p)

### Model

In [ ]:
#nn_linear = dense_layer(size=p)

nn_linear = keras.Sequential([
    layers.Input((p,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(5, activation='relu'),
    layers.Dense(1)
])

In [ ]:
nn_linear.summary()

In [ ]:
nn_linear.layers[0].get_weights()

In [ ]:
nn_linear.layers[1].get_weights()

In [ ]:
nn_linear.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.RootMeanSquaredError(name="rmse")
    ]
)

In [ ]:
assert nn_linear(train_samples[0]).shape == train_samples[1].shape

In [ ]:
history_dense = nn_linear.fit(
    x=train_samples[0],
    y=train_samples[1],
    batch_size=10,
    epochs=500,
    validation_data=valid_samples,
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

In [ ]:
train_loss = history_dense.history['rmse']
val_loss = history_dense.history['val_rmse']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'bo-', label='Training RMSE')
plt.plot(epochs, val_loss, 'ro-', label='Validation RMSE')
plt.title('Training and Validation RMSE')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.legend()

plt.show()

## LSTM neural network

In [ ]:
def lstm_io(
    df, # Pandas DataFrame with the data
    idx:[Int], # DataFrame indedx
    col:[str], # Column name
    p:Int # Number of lags
):
    # Return an (input, output) tuple for a dense_layer model
    
    assert len(idx) == len(col)

    batch_X, batch_y = [], []
    for i in range(len(idx)):
        X = df.iloc[idx[i]:(idx[i]+p)].values
        X[np.isnan(X)] = 0
        batch_X.append(X)
        batch_y.append(df.iloc[idx[i]+p][[col[i]]].values)
    
    return np.vstack(batch_X), np.vstack(batch_y)

In [ ]:
lstm = layers.LSTM(units=64, return_sequences=True, return_state=False)
fl = layers.Flatten()

In [ ]:
train_samples[0][:5].shape

In [ ]:
nn_lstm = keras.Sequential([
    layers.LSTM(units=64, return_sequences=True, return_state=False, dropout=0.1),
    #layers.LSTM(units=1, return_sequences=False, return_state=False, dropout=0.1),
    layers.Flatten(),
    layers.Dense(15, activation='relu'),
    layers.Dense(1)
])

In [ ]:
assert nn_lstm(np.expand_dims(train_samples[0], axis=-1)).shape == train_samples[1].shape

In [ ]:
nn_lstm.summary()

In [ ]:
nn_lstm.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.RootMeanSquaredError(name="rmse")
    ]
)

In [ ]:
history_lstm = nn_lstm.fit(
    x=train_samples[0],
    y=train_samples[1],
    batch_size=10,
    epochs=500,
    validation_data=valid_samples,
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

In [ ]:
train_loss = history_lstm.history['rmse']
val_loss = history_lstm.history['val_rmse']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'bo-', label='Training RMSE')
plt.plot(epochs, val_loss, 'ro-', label='Validation RMSE')
plt.title('Training and Validation RMSE')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.legend()

plt.show()

## LSTM with other countries' inflation data

The difference between this and the previous one is that it is not only autoregressive, but also considers past data from other countries.

### Data formatting

In [ ]:
def lstm_cty_io(
    df, # Pandas DataFrame with the data
    idx:[Int], # DataFrame indedx
    col:[str], # Column name
    p:Int # Number of lags
):
    # Return an (input, output) tuple for a dense_layer model
    
    assert len(idx) == len(col)

    batch_X, batch_y = [], []
    for i in range(len(idx)):
        X = df.iloc[idx[i]:(idx[i]+p)].values
        X[np.isnan(X)] = 0
        batch_X.append(X)
        batch_y.append(df.iloc[idx[i]+p][[col[i]]].values)
    
    return np.stack(batch_X, axis=0), np.vstack(batch_y)  #np.vstack(batch_X), np.vstack(batch_y)

In [ ]:
lstm_cty_io(df=df_infl_train, idx=[3, 4, 5], col=['BR', 'BR', 'BR'], p=4)[0].shape

In [ ]:
p = 24

idx, col = random_select(df_infl_train_n, p=p, n_samples=n_samples)
train_samples = lstm_cty_io(df=df_infl_train_n, idx=idx, col=col, p=p)

idx, col = random_select(df_infl_valid_n, p=p, n_samples=n_samples)
valid_samples = lstm_cty_io(df=df_infl_valid_n, idx=idx, col=col, p=p)

In [ ]:
train_samples[0].shape

### Model

In [ ]:
nn_lstm_cty = keras.Sequential([
    layers.LSTM(units=1052, return_sequences=True, return_state=False, dropout=0.1),
    layers.LSTM(units=64, return_sequences=True, return_state=False, dropout=0.1),
    #layers.LSTM(units=1, return_sequences=False, return_state=False, dropout=0.1),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

In [ ]:
nn_lstm_cty.summary()

In [ ]:
assert nn_lstm_cty(train_samples[0]).shape == train_samples[1].shape

In [ ]:
nn_lstm_cty.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.RootMeanSquaredError(name="rmse")
    ]
)

In [ ]:
train_samples[0].shape

In [ ]:
history_lstm_cty = nn_lstm_cty.fit(
    x=train_samples[0],
    y=train_samples[1],
    batch_size=20,
    epochs=500,
    validation_data=valid_samples,
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

In [ ]:
train_loss = history_lstm_cty.history['rmse']
val_loss = history_lstm_cty.history['val_rmse']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'bo-', label='Training RMSE')
plt.plot(epochs, val_loss, 'ro-', label='Validation RMSE')
plt.title('Training and Validation RMSE')
plt.xlabel('Epochs')
plt.ylabel('RMSE')
plt.legend()

plt.show()

## Model with date features

Repeating date features (eg, day in the week, month, quarter and year, week in the month, quarter and year, month in the quarter and year, and quarter in year) can be embedded and included in the model.

In [ ]:
#nn_lstm_datefeat = layers.LSTM()

## Creating a TFT model

In [ ]:
tft = TemporalFusionTransformer(
    time_steps=12,
    input_size=20,
    output_size=4,
    category_counts=5,
    n_workers=2, # Number of multiprocessing workers

    # TFT params
    input_obs_loc=24,
    static_input_loc=24,
    known_regular_input_idx=24,
    known_categorical_input_idx=24,
    column_definition=None,
)

In [ ]:
tft.time_steps

# Test

In [ ]:
batch_size = 3
n_timesteps = 4

univariate_x = np.ones(shape=(batch_size, n_timesteps))
univariate_x = keras.ops.expand_dims(univariate_x, axis=-1)
univariate_x.shape

In [ ]:
lstm = layers.LSTM(
    units=6,
    return_sequences=True,
    return_state=False,
)

In [ ]:
transf_x = lstm(univariate_x)
transf_x.shape, keras.ops.concatenate([transf_x, transf_x], axis=1).shape

In [ ]:
transf_x.shape

# References {.unnumbered}